In [16]:
import os
import re
import shutil #directory controls
import glob
import json
import sqlparse #new as of 20200629
from jsonpath_ng import jsonpath, parse
import datetime #new as of 20210411CS
import yaml
import fnmatch
import shutil
import errno

In [151]:
#set path variables
print(os.path.abspath(''))

dbt_sources_file_path = "../dbt/mcleod_dbt/models/sources/" #dbt-sources-path
dbt_models_file_path = "../dbt/mcleod_dbt/models/" #dbt-sources-path
dbt_snapshots_file_path = "../dbt/mcleod_dbt/snapshots/" #dbt-sources-path

dataform_output_sources_path = "_dataform_output/sources"
dataform_output_models_path = "_dataform_output"




#20210411CS - added to use in the template creation dates with auto generated
dateYYYYMMDD = datetime.datetime.now().strftime("%Y%m%d") #.strftime("%Y%m%d-%H-%M-%S")

C:\Users\Guilherme Alcantara\OneDrive\brf consulting\dbt - dataform converter


In [156]:
#creating directory
isExist = os.path.exists(dataform_output_sources_path)
if not isExist:
    os.makedirs(dataform_output_sources_path)
else:
    #delete all files from target
    shutil.rmtree(dataform_output_sources_path)

In [157]:
#find files on directories and subdirectories
def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename

In [158]:
# list of all .yaml files in a directory
source_files = find_files(dbt_sources_file_path, '*.yml')

#YAML loader
def read_yaml_file(filename):
    with open(filename, 'r') as stream:
        try:
            yaml_dict = yaml.load(stream, Loader=yaml.FullLoader)
            return yaml_dict
        except yaml.YAMLError as exc:
            print(exc)

#YAML -> SQLx conerter
def create_sqlx_json_source_file(source_file_path):


    for file in source_file_path:
        #reading all YAML files on source directory
        dic = read_yaml_file(file)
        #getting only sources entity from yaml file
        sources = dic["sources"]
        
        for source in sources:
            #declaring variables for each element of dictionary that will be used. tables = list, schema and database = str
            tables = source["tables"]
            database = source["database"]
            schema = source["schema"]
            #iterating through all tables for each schema
            for tables in tables:
                    #getting tables from list
                    tables= tables['name']
                    #creating json .sqlx structure
                    jmodel = "{\n"
                    jmodel += f'"type": "declaration",\n'                
                    jmodel += f'"database": "{database}",\n'
                    jmodel += f'"schema": "{schema}",\n'
                    jmodel += f'"name": "{tables}"'
                    jmodel += "\n}" #close out JSON file EOF

                    #parsing json and replacing double quotes for single quotes on keys
                    parsed = json.dumps(json.loads(jmodel), sort_keys=False, indent=2).replace('"type":','type:').replace('"database":','database:').replace('"schema":','schema:').replace('"name":','name:')

                    #creating .sqlx files
                    #write out to file in the appropriate location defined in variables
                    with open(f'{dataform_output_sources_path}/{schema}_{tables}.sqlx', "w") as jmodel_file:
                        #jmodel_file.write( script_header_template )
                        jmodel_file.write("config "+parsed)
                        jmodel_file.close()


In [159]:
create_sqlx_json_source_file(source_files)

In [160]:
def create_sqlx_models_files(model_files_path):
    for filename in model_files_path:  
        single_file_name = os.path.basename(filename)
        destination_path = (dataform_output_models_path+(filename[filename.find('models')+len('models'):filename.rfind('\\')]))+'/'
        destination_full_path = (destination_path+(os.path.basename(filename)).replace("sql","sqlx"))
        
        
        #print(destination_path)
        #print(single_file_name)
        #print(destination_full_path)
     #######copying files to new directory##############   
        try:
            #copy files if directory already exists
            
            shutil.copyfile(filename,destination_full_path)
        except IOError as e:
            # ENOENT(2): file does not exist, raised also on missing dest parent dir
            if e.errno != errno.ENOENT:
                raise
            # create directory if not exists
            
            os.makedirs(destination_path)
            # copy source files to new directory
            
            shutil.copyfile(filename,destination_full_path)
    #######end copying files to new directory##############

    #######reading file and replacing model syntax differences########
        # Read in the file
        with open(destination_full_path, 'r') as file :
            filedata = file.read()

            #getting header data to be replaced
            header=filedata[filedata.find("{{"):filedata.find("}}")+2]
            header_old=header


            header_replace_dict = {"{{":"", "}}":"}", "materialized":"type","=":":","(":"{",")":"","'":'"'}

            #iterate through dictionary keys
            for key in header_replace_dict.keys():
                #replacing all headers
                header = header.replace(key, header_replace_dict[key])         


        #writing file with replaced header and models references
        with open(destination_full_path, 'w') as file:
            new_file = filedata.replace(header_old,header)
            new_file = re.sub(r"[{][{][ ]{0,6}(ref|REF|Ref|SOURCE|source|Source)[ ]{0,9}[(][ ]{0,9}[']?",'${ref("', new_file)            
            new_file = re.sub(r"([ ]{0,9}[_'][.]{0,10}[ ]{0,20}(,)[.]{0,20}[ ]{0,20}[.]{0,20}['])",'_',new_file)
            new_file = re.sub(r"(['][)][ ]{0,9}[}])",'")',new_file)

            try:
                file.write(new_file)                
                print('Generated file: '+destination_full_path)
            except OSError as error : 
                print(error) 
    #######end reading file and replacing model syntax differences########
        
           
        



In [161]:
# list of all .yaml files in a directory
model_files = find_files(dbt_models_file_path, '*.sql')

create_sqlx_models_files(model_files)

Generated file: _dataform_output/master/a_data_count_validations.sqlx
Generated file: _dataform_output/master/w_call_record_d.sqlx
Generated file: _dataform_output/master/w_destination_stop_d.sqlx
Generated file: _dataform_output/master/w_driver_f.sqlx
Generated file: _dataform_output/master/w_edi_orders_d.sqlx
Generated file: _dataform_output/master/w_equipment_lkp.sqlx
Generated file: _dataform_output/master/w_movement_d.sqlx
Generated file: _dataform_output/master/w_movement_f.sqlx
Generated file: _dataform_output/master/w_open_item_d.sqlx
Generated file: _dataform_output/master/w_open_item_lkp.sqlx
Generated file: _dataform_output/master/w_order_detail_ext_f.sqlx
Generated file: _dataform_output/master/w_order_detail_f.sqlx
Generated file: _dataform_output/master/w_origin_stop_d.sqlx
Generated file: _dataform_output/master/w_other_charge_lkp.sqlx
Generated file: _dataform_output/master/w_segment_d.sqlx
Generated file: _dataform_output/master/w_tractor_movement_d.sqlx
Generated file